# データサイエンス応用基礎 3.6章「認識」ソースコード
- google colabでも動作するようにしています．google colabで動かす場合は右のボタンをクリックしてください．
[![google colabで開く](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/schwalbe1996/ds/blob/main/source.ipynb)
- google colabで動かす場合は，「ランタイム」⇒「ランタイムのタイプを変更」からハードウェアアクセラレータを有効（CPU以外）にすることをお勧めします．（CPUでも動作しますが処理速度が非常に遅いです)


In [5]:
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from torch import nn

# GPUが使える場合はGPUを使うように設定します．
device = "cuda" if torch.cuda.is_available() else "cpu"

In [6]:
# MNISTデータセットを自動的にダウンロードして用意します．

batch_size=32 # ミニバッチのサイズをここで指定してます

training_data = datasets.MNIST('data', train=True, download=True, transform=ToTensor())
test_data = datasets.MNIST('data', train=False, download=True, transform=ToTensor())
train_dataloader = DataLoader(training_data, batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size)


In [7]:
# 畳み込みニューラルネットワークのモデル
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        # 最初の畳み込み層＋Maxプーリング層
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, 5),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        # 2番目の畳み込み層+Maxプーリング層
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        # 2番目のプーリング層の出力（16チャンネル4x4）を入力とした全結合層．出力は120次元
        self.fc1 = nn.Sequential(
            nn.Linear(16*4*4, 120),
            nn.ReLU()
        )
        # 2つめの全結合層．入力は120次元，出力は84次元
        self.fc2 = nn.Sequential(
            nn.Linear(120, 84),
            nn.ReLU()
        )
        # 最後の全結合層．入力は84次元，出力は（クラス数と同じ）10次元
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        return self.fc3(out)

In [ ]:
model = MyModel().to(device) # 上で記述したモデルを使います
loss_func = nn.CrossEntropyLoss() # 損失関数にクロスエントロピーを用いる
optimizer = torch.optim.Adam(model.parameters(), lr=1.0e-4) # 最適化手法にAdamを用いる

epochs = 10 # エポック数の設定．学習データをepochs回繰り返し用いる．
for epoch in range(epochs):
    train_loss = 0
    test_loss = 0
    correct = 0
    # 学習部分
    model.train()
    for i, (X,y) in enumerate(train_dataloader):
        X = X.to(device)
        y = y.to(device)

        model.zero_grad()
        y_pred = model(X)
        loss = loss_func(y_pred, y)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_loss /= len(train_dataloader)
    # テストデータを用いて損失と正解率を計算
    model.eval()
    for i, (X,y) in enumerate(test_dataloader):
        X = X.to(device)
        y = y.to(device)

        y_pred = model(X)
        loss = loss_func(y_pred, y)
        test_loss += loss.item()
        correct += (y_pred.argmax(dim=1) == y).sum().item()

    test_loss /= len(test_dataloader)
    correct /= len(test_dataloader.dataset)
    print('Epoch:', epoch, 'Train Loss:', train_loss, 'Test Loss:', test_loss, 'Accuracy:', correct)

In [ ]:
# テスト用データセットから1バッチ分だけデータを読み込んできて予測するコード
x,y = next(iter(test_dataloader)) # 1バッチ分だけデータを読み込んでくる
model.eval()
x = x.to(device)
pred = model(x).argmax(dim=1).cpu() # 予測部分
print(f"正解＝{y[0:10]}") # 最初の10個の正解ラベルを出力
print(f"予測={pred[0:10]}") # 最初の10個の予測を出力

In [10]:
# 学習済のモデルを保存するコード
torch.save(model.state_dict(), "model.pth")

In [13]:
# 学習済のモデルを読み込む場合
model = MyModel() # 「class MyModel(nn.Module)」のコードを実行しておく必要があります
model.load_state_dict(torch.load("model.pth"))
model = model.to(device)